In [1]:
import multiprocessing
import time
import numpy as np
import scipy as sp
import emcee
import testkit
import ciamod
import os
import gc
import sys
import pickle
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# set up the model arguments the drop these into theta(state vector) or runargs

# set up pressure grids in bar cos its intuitive
logcoarsePress = np.arange(-4.0, 2.5, 0.53)
#logcoarsePress = np.arange(-4.0, 3.0, 0.5)
coarsePress = 1000.* pow(10,logcoarsePress)
logfinePress = np.arange(-4.0, 2.4, 0.1)
finePress = 1000.* pow(10,logfinePress)
# forward model wants pressure in mbar
press = finePress
nprof = coarsePress.size
print nprof
print finePress.size

13
64


In [3]:
#r2d2 = 1.
#logg = 5.0
#dlam = 0.
w1 = 1.0
w2 = 2.5
pcover = 1.0
do_clouds = 0
use_disort = 0 

# now the linelist
# Set up number of gases, and point at the lists. see gaslist.dat
ngas = 8
gasnum = np.asfortranarray(np.array([1,2,4,5,6,3,20,21],dtype='i'))
lists = ["../Linelists/xsecarrH2O_1wno_500_10000.save","../Linelists/xsecarrCH4_1wno_500_10000.save","../Linelists/xsecarrCO_1wno_500_10000_02.save","../Linelists/xsecarrCO2_1wno_500_10000_02.save" ,"../Linelists/xsecarrNH3_1wno_500_10000.save","../Linelists/xsecarrH2S_1wno_500_10000.save","../Linelists/xsecarrK_new_1wno_500_10000_02.save","../Linelists/xsecarrNa_new_1wno_500_10000_02.save"]


In [4]:
# get the basic framework from water list
x=readsav('../Linelists/xsecarrH2O_1wno_500_10000.save')
inlinelist=x.xsecarr  #3D array with Nwavenubmers x Ntemps x Npressure
inlinetemps=np.asfortranarray(x.t,dtype='float64')
inpress=1000.*x.p
rawwavenum=x.wno
wn1 = 10000./w2
wn2 = 10000. / w1
inwavenum = np.asfortranarray(rawwavenum[np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1)))],dtype='float64')
ntemps = inlinetemps.size
npress= finePress.size
nwave = inwavenum.size
r1 = np.amin(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))
r2 = np.amax(np.where(np.logical_not(np.logical_or(rawwavenum[:] > wn2, rawwavenum[:] < wn1))))

# Here we are interpolating the linelist onto our fine pressure scale. 
linelist = (np.ones([ngas,npress,ntemps,nwave],order='F')).astype('float64', order='F')
for gas in range (0,ngas):
    inlinelist=readsav(lists[gas]).xsecarr
    for i in range (0,ntemps):
        for j in range (r1,r2+1):
            pfit = interp1d(np.log10(inpress),np.log10(inlinelist[:,i,j]))
            linelist[gas,:,i,(j-r1)] = np.asfortranarray(pfit(np.log10(finePress)))

linelist[np.isnan(linelist)] = -50.0


In [5]:
# cloudparams is structured array with 5 entries
# each one has a patch*cloud entries
cloudparams = np.ones(5)
# 5 entries in cloudparams for simple slab model are:
# 0) log10(number density)
# 1) top layer id (or pressure)
# 2) base ID (these are both in 61 layers)
# 3) rg
# 4) rsig
cloudparams[0] = -20.
cloudparams[1] = 10
cloudparams[2] = 12
cloudparams[3] = 1e-4
cloudparams[4] = 1e-5
# hardwired gas and cloud IDs
cloudnum = np.array([1],dtype='i')



In [6]:
# Get the cia bits
tmpcia, ciatemps = ciamod.read_cia("CIA_DS_aug_2015.dat",inwavenum)
cia = np.asfortranarray(np.empty((4,ciatemps.size,nwave)),dtype='float32')
cia[:,:,:] = tmpcia[:,:,:nwave] 
ciatemps = np.asfortranarray(ciatemps, dtype='float32')


In [7]:
# hardwired FWHM of data
fwhm = 0.013
#fixvmrs = -8.0

# get the observed spectrum
obspec = np.asfortranarray(np.loadtxt("G570D_calib.dat",dtype='d',unpack='true'))

# NEXT LINE ADDS SYTEMATIC TO SPECTRUM FOR Log F retrieval
#obspec[1,:] = obspec[1,:] + 0.1*(min(obspec[2,10::3]**2))


runargs = pcover, cloudparams,do_clouds,gasnum,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec


In [11]:
ndim = 24
theta = np.empty(ndim)
theta[0] = np.random.randn() - 3.5 # H2O
theta[1] = np.random.randn() - 3.5 # CH4
theta[2] = np.random.randn() - 7.5 # CO
theta[3] = np.random.randn() - 8.0 # CO2
theta[4] = np.random.randn() - 4.6 # NH3
theta[5] = np.random.randn() - 9.0  #H2S
theta[6] = np.random.randn() - 5.5  # Na+K
theta[7] = np.random.rand() + 4.0
theta[8] =  1.5e-19 + (np.random.randn() * 1.e-20)
theta[9] = np.random.randn() * 0.01
#p0[:,5] = np.log10((np.random.rand(nwalkers).reshape(nwalkers) * (min(obspec[2,10::3]**2)*(0.1 - 0.001))) + (0.001*min(obspec[2,10::3]**2)))
theta[10] =  50. + (np.random.randn())
#p0[:,6] = (-2. *  np.random.rand(nwalkers).reshape(nwalkers)) - 2.
#p0[:,6] = 200. + ( np.random.rand(nwalkers).reshape(nwalkers) * 100. )
toffset = 150.* np.random.rand()
for i in range (11,10+nprof):
    #p0[:,i] = p0[:,6] + (150.*(i-6))
    theta[i] = 650.+toffset + ((coarsePress[i-11]/100.)**1.1)
theta[23] = 3000. + 200 * np.random.rand()

In [16]:
theta2 = np.array([-2.64174651e+00,  -2.92174198e+00,  -1.06520064e+01,  -6.85512214e+00,
   -7.71799871e+00,  -1.16581345e+01,  -4.53163570e+00,   5.97976476e+00,
    2.47941068e-20,  -1.94873539e-03,   9.05698433e+02,   9.38519853e+02,
    9.38521271e+02,   9.38526697e+02,   9.38547471e+02,   9.38626999e+02,
    9.38931450e+02,   9.40096963e+02,   9.44558838e+02,   9.61640000e+02,
    1.02703091e+03,   1.27736351e+03,   2.23569862e+03,   2.97941114e+03])

print theta
print theta2

[ -2.95457004e+00  -3.97258331e+00  -8.86151548e+00  -8.87761641e+00
  -4.92530567e+00  -1.04515140e+01  -6.89595704e+00   4.70991722e+00
   1.35977921e-19   7.82725714e-03   4.99305213e+01   7.96631731e+02
   7.96633149e+02   7.96638575e+02   7.96659349e+02   7.96738876e+02
   7.97043327e+02   7.98208841e+02   8.02670716e+02   8.19751878e+02
   8.85142791e+02   1.13547539e+03   2.09381050e+03   3.18688346e+03]
[ -2.64174651e+00  -2.92174198e+00  -1.06520064e+01  -6.85512214e+00
  -7.71799871e+00  -1.16581345e+01  -4.53163570e+00   5.97976476e+00
   2.47941068e-20  -1.94873539e-03   9.05698433e+02   9.38519853e+02
   9.38521271e+02   9.38526697e+02   9.38547471e+02   9.38626999e+02
   9.38931450e+02   9.40096963e+02   9.44558838e+02   9.61640000e+02
   1.02703091e+03   1.27736351e+03   2.23569862e+03   2.97941114e+03]


In [18]:
print testkit.lnprob(theta,*runargs) * -2. / (obspec.shape[1] /3.)
print testkit.lnprob(theta2,*runargs) * -2. / (obspec.shape[1] /3.)


[[  2.50000000e+00   2.49937516e+00   2.49875062e+00 ...,   1.00020004e+00
    1.00010001e+00   1.00000000e+00]
 [  4.02828419e-16   4.00673178e-16   4.00611870e-16 ...,   1.80113599e-15
    1.75839921e-15   1.21721245e-15]]
1.5
[[  2.50782726e+00   2.50720241e+00   2.50657788e+00 ...,   1.00802730e+00
    1.00792727e+00   1.00782726e+00]
 [  4.02828419e-16   4.00673178e-16   4.00611870e-16 ...,   1.80113599e-15
    1.75839921e-15   1.21721245e-15]]
53
243.550065827
[[  2.50000000e+00   2.49937516e+00   2.49875062e+00 ...,   1.00020004e+00
    1.00010001e+00   1.00000000e+00]
 [  2.33574177e-16   2.34741464e-16   2.31964092e-16 ...,   1.26069454e-15
    1.25846352e-15   1.18636677e-15]]
1.5
[[  2.49805126e+00   2.49742642e+00   2.49680189e+00 ...,   9.98251305e-01
    9.98151275e-01   9.98051265e-01]
 [  2.33574177e-16   2.34741464e-16   2.31964092e-16 ...,   1.26069454e-15
    1.25846352e-15   1.18636677e-15]]
53
-38.1202968738
